### Perform SQL Queries Using SQLite on Local Machine

In [1]:
# Import packages

import sqlite3
import pandas as pd

In [2]:
# Check SQLite version

print("SQLite Version is:", sqlite3.sqlite_version)

SQLite Version is: 3.33.0


In [3]:
# Create connection to dbfile --> dbfile is in the same working directory as the Jupyter notebook

dbfile = 'sqlite_db_pythonsqlite.db'
conn = sqlite3.connect(dbfile)

In [4]:
# Create cursor

cur = conn.cursor()

In [5]:
# Read all tables names

table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

# print table list
print(table_list)

[('Bookings',), ('Facilities',), ('Members',)]


### QUESTIONS

#### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that charge a members fee. (Optional: add fee amount and order by fee in descending order)

In [6]:
# Q1 Answer --> Part1: Examine Facilities table by selecting all

result = pd.read_sql_query("""SELECT * 
                               FROM Facilities""", conn)
print(result)

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


In [7]:
# Q1 Answer --> Part2: Use WHERE membercost > 0

result = pd.read_sql_query("""SELECT name, membercost 
                                FROM Facilities 
                                WHERE membercost > 0 
                                ORDER BY membercost DESC""", conn)
print(result)

             name  membercost
0  Massage Room 1         9.9
1  Massage Room 2         9.9
2  Tennis Court 1         5.0
3  Tennis Court 2         5.0
4    Squash Court         3.5


#### Q2: How many facilities do not charge a fee to members?

In [8]:
# Q2 Answer

result = pd.read_sql_query("""SELECT COUNT(facid) AS NUM_of_FAC_with_MEMBERS_FEE 
                                FROM Facilities 
                                WHERE membercost = 0""", conn)
print(result)

   NUM_of_FAC_with_MEMBERS_FEE
0                            4


#### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question. (Optional: order by memebers fee)

In [9]:
# Q3 Answer

result = pd.read_sql_query("""SELECT facid, name, membercost, monthlymaintenance 
                                FROM Facilities 
                                WHERE membercost > 0 AND membercost < 0.2 * monthlymaintenance 
                                ORDER BY membercost DESC""", conn)
print(result)

   facid            name  membercost  monthlymaintenance
0      4  Massage Room 1         9.9                3000
1      5  Massage Room 2         9.9                3000
2      0  Tennis Court 1         5.0                 200
3      1  Tennis Court 2         5.0                 200
4      6    Squash Court         3.5                  80


#### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [10]:
# Q4 Answer

result = pd.read_sql_query("""SELECT * 
                                FROM Facilities 
                                WHERE facid in (1, 5)""", conn)
print(result)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


#### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than 100 USD . Return the name and monthly maintenance of the facilities in question.

In [11]:
# Q5 Answer

# Cheap
result = pd.read_sql_query("""SELECT name AS Facility_Name, 
                                        monthlymaintenance, 
                                        CASE WHEN monthlymaintenance <= 100 THEN 'Cheap' 
                                        ELSE 'Expensive' 
                                        END AS Category 
                                FROM Facilities 
                                ORDER BY monthlymaintenance DESC""", conn)
print(result)

     Facility_Name  monthlymaintenance   Category
0   Massage Room 1                3000  Expensive
1   Massage Room 2                3000  Expensive
2   Tennis Court 1                 200  Expensive
3   Tennis Court 2                 200  Expensive
4     Squash Court                  80      Cheap
5  Badminton Court                  50      Cheap
6    Snooker Table                  15      Cheap
7       Pool Table                  15      Cheap
8     Table Tennis                  10      Cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution. 

In [12]:
# Q6 Answer --> Part1: Examine Members table by selecting all

result = pd.read_sql_query("""SELECT *
                                FROM Members
                                LIMIT 10""", conn)
print(result)

   memid   surname firstname                             address  zipcode  \
0      0     GUEST     GUEST                               GUEST        0   
1      1     Smith    Darren          8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy        8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim              23 Highway Way, Boston    23423   
4      4  Joplette    Janice          20 Crossing Road, New York      234   
5      5   Butters    Gerald      1065 Huntingdon Avenue, Boston    56754   
6      6     Tracy    Burton             3 Tunisia Drive, Boston    45678   
7      7      Dare     Nancy         6 Hunting Lodge Way, Boston    10383   
8      8    Boothe       Tim  3 Bloomsbury Close, Reading, 00234      234   
9      9  Stibbons    Ponder           5 Dragons Way, Winchester    87630   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-0

In [13]:
# Q6 Answer --> Use MAX(joindate) to avoid any mistakes
# Alternatevely can use MAX(memid) which assumes that memid are issued in order
# Note that MAX(x) cannot be used directly with WHERE clause, but need to be included in a SELECT statement as written below

result = pd.read_sql_query("""SELECT firstname, surname
                                FROM Members
                                WHERE joindate = (SELECT MAX(joindate) FROM Members)""", conn) 
print(result)

  firstname surname
0    Darren   Smith


#### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [14]:
# Q7 Answer --> Part1: Examine Bookings table by selecting all

result = pd.read_sql_query("""SELECT *
                                FROM Bookings
                                LIMIT 10""", conn)
print(result)

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1
5       5      8      1  2012-07-03 15:00:00      1
6       6      0      2  2012-07-04 09:00:00      3
7       7      0      2  2012-07-04 15:00:00      3
8       8      4      3  2012-07-04 13:30:00      2
9       9      4      0  2012-07-04 15:00:00      2


In [15]:
# Q7 Answer --> Part2: Examine Facilities table by selecting all

result = pd.read_sql_query("""SELECT *
                                FROM Facilities""", conn)
print(result)

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


In [16]:
# Q7 Answer --> Part3: Select distinct memeber names which have made bookings for facilities with facid = 0 and 1

result = pd.read_sql_query("""SELECT DISTINCT Members.firstname || ' ' || Members.surname AS Member_Name, 
                                                Facilities.name AS Facility_Name
                                FROM Members
                                INNER JOIN Bookings ON Members.memid = Bookings.memid
                                INNER JOIN Facilities ON Facilities.facid = Bookings.facid
                                WHERE Facilities.facid in (0, 1)
                                ORDER BY Members.firstname, Members.surname, Facilities.name""", conn)
print(result)

          Member_Name   Facility_Name
0          Anne Baker  Tennis Court 1
1          Anne Baker  Tennis Court 2
2        Burton Tracy  Tennis Court 1
3        Burton Tracy  Tennis Court 2
4        Charles Owen  Tennis Court 1
5        Charles Owen  Tennis Court 2
6        Darren Smith  Tennis Court 2
7       David Farrell  Tennis Court 1
8       David Farrell  Tennis Court 2
9         David Jones  Tennis Court 1
10        David Jones  Tennis Court 2
11       David Pinker  Tennis Court 1
12      Douglas Jones  Tennis Court 1
13      Erica Crumpet  Tennis Court 1
14     Florence Bader  Tennis Court 1
15     Florence Bader  Tennis Court 2
16        GUEST GUEST  Tennis Court 1
17        GUEST GUEST  Tennis Court 2
18     Gerald Butters  Tennis Court 1
19     Gerald Butters  Tennis Court 2
20   Henrietta Rumney  Tennis Court 2
21         Jack Smith  Tennis Court 1
22         Jack Smith  Tennis Court 2
23    Janice Joplette  Tennis Court 1
24    Janice Joplette  Tennis Court 2
25     Jemim

#### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than 30 USD. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [17]:
# Q8 Answer

result = pd.read_sql_query("""SELECT Members.firstname || ' ' || Members.surname AS Full_Name, 
                                        Facilities.name AS Facility_Name,
                                        CASE WHEN Members.memid = 0 THEN SUM(Bookings.slots) * Facilities.guestcost 
                                            ELSE SUM(Bookings.slots) * Facilities.membercost 
                                            END AS Total_Cost
                                FROM Members
                                INNER JOIN Bookings ON Members.memid = Bookings.memid
                                INNER JOIN Facilities ON Facilities.facid = Bookings.facid
                                WHERE date(Bookings.starttime) = '2012-09-14'
                                GROUP BY Members.memid, Facilities.facid
                                HAVING Total_Cost > 30
                                ORDER BY Total_Cost DESC""", conn)
print(result)

        Full_Name   Facility_Name  Total_Cost
0     GUEST GUEST  Massage Room 1       480.0
1     GUEST GUEST  Massage Room 2       320.0
2     GUEST GUEST  Tennis Court 2       225.0
3     GUEST GUEST  Tennis Court 1       150.0
4     GUEST GUEST    Squash Court       140.0
5  Jemima Farrell  Massage Room 1        59.4


#### Q9: This time, produce the same result as in Q8, but using a subquery. 

In [18]:
# Q9 Answer --> Replace the CASE statement with SELECT IIF(X, Y, Z) which returns Y if X is TRUE and Z if X is FALSE
# Note: The iif() function was introduced in SQLite 3.32.0, which was released on 22 May 2020 --> 
        # iif() does not work for earlier SQLite3 versions  

result = pd.read_sql_query("""SELECT Members.firstname || ' ' || Members.surname AS Full_Name, 
                                        Facilities.name AS Facility_Name,
                                        (SELECT IIF(Members.memid = 0, 
                                                    SUM(Bookings.slots) * Facilities.guestcost,  
                                                    SUM(Bookings.slots) * Facilities.membercost)) 
                                        AS Total_Cost 
                                FROM Members
                                INNER JOIN Bookings ON Members.memid = Bookings.memid
                                INNER JOIN Facilities ON Facilities.facid = Bookings.facid
                                WHERE date(Bookings.starttime) = '2012-09-14'
                                GROUP BY Members.memid, Facilities.facid
                                HAVING Total_Cost > 30
                                ORDER BY Total_Cost DESC""", conn)
print(result)

        Full_Name   Facility_Name  Total_Cost
0     GUEST GUEST  Massage Room 1       480.0
1     GUEST GUEST  Massage Room 2       320.0
2     GUEST GUEST  Tennis Court 2       225.0
3     GUEST GUEST  Tennis Court 1       150.0
4     GUEST GUEST    Squash Court       140.0
5  Jemima Farrell  Massage Room 1        59.4


#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [19]:
# Q10 Answer

result = pd.read_sql_query("""SELECT Facilities.name AS Facility_Name,
                                        (SELECT IIF(Members.memid = 0, 
                                                    SUM(Bookings.slots) * Facilities.guestcost,  
                                                    SUM(Bookings.slots) * Facilities.membercost))  
                                        AS Total_Revenue
                                FROM Facilities
                                INNER JOIN Bookings ON Facilities.facid = Bookings.facid
                                INNER JOIN Members ON Members.memid = Bookings.memid 
                                GROUP BY Facilities.facid
                                HAVING Total_Revenue < 1000
                                ORDER BY Total_Revenue DESC, Facility_Name""", conn)
print(result)

     Facility_Name  Total_Revenue
0  Badminton Court              0
1       Pool Table              0
2    Snooker Table              0
3     Table Tennis              0


In [20]:
# Q10 Answer Comment 
    # Appears that there are no guest bookings for Badminton Court, Pool Table, Snooker Table and Table Tennis
    # Member's cost is 0 for these facilities
    # Check to see if there will be Total_Revenue > 0 by increasing revenue limit to 10k

result = pd.read_sql_query("""SELECT Facilities.name AS Facility_Name,
                                        (SELECT IIF(Members.memid = 0, 
                                                    SUM(Bookings.slots) * Facilities.guestcost,  
                                                    SUM(Bookings.slots) * Facilities.membercost))  
                                        AS Total_Revenue
                                FROM Facilities
                                INNER JOIN Bookings ON Facilities.facid = Bookings.facid
                                INNER JOIN Members ON Members.memid = Bookings.memid 
                                GROUP BY Facilities.facid
                                HAVING Total_Revenue < 10000 
                                ORDER BY Total_Revenue DESC, Facility_Name""", conn)
print(result)

     Facility_Name  Total_Revenue
0   Tennis Court 1           6600
1  Badminton Court              0
2       Pool Table              0
3    Snooker Table              0
4     Table Tennis              0


#### Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [21]:
# Q11 Answer

result = pd.read_sql_query("""SELECT m1.surname, 
                                        m1.firstname, 
                                        m2.firstname || ' ' || m2.surname AS Recommended_by 
                                FROM Members AS m1 
                                INNER JOIN Members AS m2 ON m2.memid = m1.recommendedby
                                ORDER BY m1.surname, m1.firstname""", conn)
print(result)

              surname  firstname     Recommended_by
0               Bader   Florence    Ponder Stibbons
1               Baker       Anne    Ponder Stibbons
2               Baker    Timothy     Jemima Farrell
3              Boothe        Tim         Tim Rownam
4             Butters     Gerald       Darren Smith
5              Coplin       Joan      Timothy Baker
6             Crumpet      Erica        Tracy Smith
7                Dare      Nancy    Janice Joplette
8             Genting    Matthew     Gerald Butters
9                Hunt       John  Millicent Purview
10              Jones      David    Janice Joplette
11              Jones    Douglas        David Jones
12           Joplette     Janice       Darren Smith
13          Mackenzie       Anna       Darren Smith
14               Owen    Charles       Darren Smith
15             Pinker      David     Jemima Farrell
16            Purview  Millicent        Tracy Smith
17             Rumney  Henrietta    Matthew Genting
18          

#### Q12: Find the facilities with their usage by member, but not guests 

In [22]:
# Q12 Answer

result = pd.read_sql_query("""SELECT Members.firstname || ' ' || Members.surname AS Member_Name, 
                                        Facilities.name AS Facility_Name, 
                                        SUM(Bookings.slots) AS Facility_Usage_by_Member
                                FROM Members
                                INNER JOIN Bookings ON Members.memid = Bookings.memid
                                INNER JOIN Facilities ON Facilities.facid = Bookings.facid
                                WHERE Members.memid > 0
                                GROUP BY Members.memid
                                ORDER BY Facility_Usage_by_Member DESC, Member_Name""", conn)
print(result)

                Member_Name    Facility_Name  Facility_Usage_by_Member
0              Darren Smith     Table Tennis                       685
1                Tim Rownam   Massage Room 1                       660
2                Tim Boothe   Tennis Court 2                       440
3               Tracy Smith   Tennis Court 1                       435
4            Gerald Butters   Tennis Court 1                       409
5              Burton Tracy   Tennis Court 2                       366
6              Charles Owen   Tennis Court 1                       345
7           Janice Joplette   Massage Room 1                       326
8               David Jones   Tennis Court 2                       305
9                Anne Baker   Tennis Court 1                       296
10            Timothy Baker   Tennis Court 2                       290
11               Nancy Dare  Badminton Court                       267
12          Ponder Stibbons   Tennis Court 2                       249
13    

#### Q13: Find the facilities usage by month, but not guests 

In [23]:
# Q13 Answer

result = pd.read_sql_query("""SELECT Facilities.name AS Facility_Name, 
                                        strftime('%m', date(starttime)) AS Month, 
                                        SUM(Bookings.slots) AS Bookings_by_Month
                                FROM Bookings
                                INNER JOIN Facilities ON Facilities.facid = Bookings.facid
                                INNER JOIN Members ON Members.memid = Bookings.memid
                                WHERE Members.memid > 0
                                GROUP BY Facilities.facid, Month
                                ORDER BY Facility_Name, Month""", conn)
print(result)

      Facility_Name Month  Bookings_by_Month
0   Badminton Court    07                165
1   Badminton Court    08                414
2   Badminton Court    09                507
3    Massage Room 1    07                166
4    Massage Room 1    08                316
5    Massage Room 1    09                402
6    Massage Room 2    07                  8
7    Massage Room 2    08                 18
8    Massage Room 2    09                 28
9        Pool Table    07                110
10       Pool Table    08                303
11       Pool Table    09                443
12    Snooker Table    07                140
13    Snooker Table    08                316
14    Snooker Table    09                404
15     Squash Court    07                 50
16     Squash Court    08                184
17     Squash Court    09                184
18     Table Tennis    07                 98
19     Table Tennis    08                296
20     Table Tennis    09                400
21   Tenni

In [24]:
cur.close()
conn.close()